In [2]:
import tweepy
from tweepy import OAuthHandler
from get_config import get_config

env = get_config()

consumer_key = env.get('CONSUMER_KEY')
consumer_secret = env.get('CONSUMER_SECRET')
access_token = env.get('ACCESS_TOKEN')
access_secret = env.get('ACCESS_TOKEN_SECRET')
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [3]:
# pull out 200 most recent tweets
# not sure what the page parameter does (but it does affect the output)
def one_search():
    tweets = api.user_timeline(id="@ThisIsFusion",count=200,page=10,include_rts=True)
    all_items=[]
    [all_items.append(i) for i in tweets]
    return all_items

tweets = one_search()

In [4]:
# the most n retweets in the 200 recent tweets
def most_retweets(n):
    d = {}
    for item in tweets:
        try:
            # filter out retweeted content
            if item.retweeted_status:
                pass
        except:
            d[item.id] = item.retweet_count
    return sorted(d.items(), key=lambda x: x[1], reverse=True)[:n]

most = most_retweets(5)
print "The 5 most retweets\n"
print "      id          frequency"
for i in most:
    print i

The 5 most retweets

      id          frequency
(614807708575907840, 1208)
(615204562715213824, 120)
(614533777159798784, 104)
(614877330025119744, 81)
(615639709025464320, 53)


In [8]:
# first 100 retweet user id list for the nth most retweet
def nth_retweet_users(n):
    retweets = api.retweets(most[n][0], 100)
    users = []
    for i in range(len(retweets)):
        users.append(retweets[i].user.id)
    return users

# text for the nth most retweet
def nth_retweet_text(n):
    return api.retweets(most[n][0])[0].text

# most retweet
users = nth_retweet_users(0)
text = nth_retweet_text(0)

print 'The most retweeted text:'
print text + '\n'
print 'User ids associated with the %d retweets:' % len(users)
print users

The most retweeted text:
RT @ThisIsFusion: People tweet #FreeBree in support of the black woman who removed S.C.’s Confederate flag http://t.co/n2DtA16cRn http://t.…

User ids associated with the 91 retweets:
[16877020, 2453868779, 97781259, 1950028297, 1681126010, 2770725958, 2339364644, 206264653, 2213807803, 2310771122, 3096761546, 2725212535, 2803939900, 199218897, 1093933434, 2950787838, 849887262, 870667650, 2884916357, 82774113, 3313694963, 3255993631, 855561139, 604726301, 237719164, 122039252, 620616767, 1643437861, 220251523, 32546732, 1676473459, 2252121918, 3077784330, 19012145, 2917748498, 26249813, 535801357, 979514185, 2450404470, 3244592486, 88374158, 1529312899, 27029718, 2844628040, 3225700620, 2329547336, 1585880785, 45885400, 97676329, 117393456, 345616616, 2392323869, 254082700, 1015909742, 256591329, 1062292766, 746607206, 1374133771, 217677533, 66187633, 331166340, 1391643078, 19569085, 3104869754, 586062902, 49976556, 378514674, 2262126988, 2658812380, 125633058